In [2]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from hrvanalysis import get_time_domain_features, get_frequency_domain_features
from hrvanalysis.preprocessing import get_nn_intervals
import biosppy.signals.ecg as ecg
from tqdm import tqdm
import neurokit2 as nk
from sklearn.utils import class_weight
import json
import pyhrv.tools as tools
from pyhrv.hrv import hrv

sys.path.append('..')
from util import load_base_data

from pdb import set_trace

import warnings
warnings.simplefilter("ignore")

plt.rcParams['figure.figsize'] = [20, 5]

In [3]:
SAMPLING_RATE = 300
X_train, y_train, X_test = load_base_data(just_train=False, standard=True, flip=True)


In [ ]:
DATA_PATH = "../data/base"
SAMPLING_RATE = 300
X_test = pd.read_csv(f'{DATA_PATH}/X_test.csv')
X_train = pd.read_csv(f'{DATA_PATH}/X_train.csv')
y_train = pd.read_csv(f'{DATA_PATH}/y_train.csv', index_col='id')

In [7]:
X_train.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x17797,x17798,x17799,x17800,x17801,x17802,x17803,x17804,x17805,x17806
id,,,,,,,,,,,,,,,,,,,,,
0,-0.203345,-0.166647,-0.139124,-0.120775,-0.084077,-0.065728,-0.029031,0.026016,0.126934,0.237027,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.318997,0.718533,1.720038,2.735953,3.723047,4.530015,5.120831,5.524315,5.855748,6.014259,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.248745,-0.258787,-0.263808,-0.273850,-0.283892,-0.288913,-0.298954,-0.308996,-0.319038,-0.334101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.474936,1.503172,1.526702,1.559643,1.606703,1.681998,1.766705,1.874942,2.006708,2.072592,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.094774,1.232391,1.332476,1.426307,1.470094,1.501371,1.526392,1.538903,1.557669,1.570179,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def read_data(dataname):
    X_test = pd.read_csv(f'../data/{dataname}/X_test.csv')
    X_train = pd.read_csv(f'../data/{dataname}/X_train.csv')
    return X_train, X_test
def concat_data(x1, x2):
    concated_x = pd.concat([x1, x2.drop(columns=['id'])], axis=1)
    return concated_x

In [4]:
def generate_interval_features(data, file_path, start=0, generate_fn=True):
    feature_names = ['nni_counter', 'nni_mean', 'nni_min', 'nni_max', 'hr_mean', 'hr_min', 'hr_max', 'hr_std', 
           'nni_diff_mean', 'nni_diff_min', 'nni_diff_max', 'sdnn', 'rmssd', 
           'sdsd', 'nn50', 'pnn50', 'nn20', 'pnn20', 'sd1', 'sd2', 'sd_ratio'] #, 'ellipse_area', 'sampen']
#     feature_vecs = []
    error_counts = 0
    
    if generate_fn:
        with open(file_path, "w") as f:
            f.write("id,"+",".join(feature_names) + "\n")
        
    for i in tqdm(range(start, len(data))):
        signal = data.loc[i].dropna().to_numpy(dtype='float32')
        feature_vector = None

        try:
            signals, rpeaks = ecg.ecg(signal, show=False)[1:3]
#             signals, info = nk.ecg_process(signal, sampling_rate=SAMPLING_RATE)
#             r_peaks = info["ECG_R_Peaks"]
            nni = tools.nn_intervals(rpeaks)
            extracted = hrv(nni, rpeaks, signals, SAMPLING_RATE)
            feature_vector = np.array([extracted[k] for k in feature_names])
        except:
            pass
        
        if feature_vector is None:
            error_counts += 1
            feature_vector = [np.nan]*len(feature_names)
#         feature_vecs.append(feature_vector)

        with open(file_path, "a") as f:
            f.write(f"{str(i)},"+",".join([str(x) for x in feature_vector])+"\n")

#     return feature_names, np.array(feature_vecs), error_counts
    return feature_names, error_counts


In [6]:
# %%capture
feature_names, error_counts = generate_interval_features(X_train, "X_train.csv", start=0, generate_fn=True)

 48%|████▊     | 2440/5117 [00:42<00:46, 57.47it/s]

KeyboardInterrupt



In [5]:
# %%capture
feature_names, error_counts = generate_interval_features(X_test, "X_test.csv")

100%|██████████| 3411/3411 [00:55<00:00, 61.30it/s]
